In [1]:
from imports import * # Catchall for commonly used imports. Will still throw linter warnings for imports not explicitly called below

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from bs4 import BeautifulSoup
import requests
import os
import re


import warnings
warnings.filterwarnings('ignore')

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)

### Acquire and process Central Alberta cattle auction data from three sites:
- vjvauction.com (Ponoka, Rimbey, and Westlock markets)
- oldsauction.com
- drylandcattle.com (Veteran market)

In [2]:
# Establish urls

vjv_ponoka_url = 'https://vjvauction.com/ponoka/market-report/cattle'
vjv_rimbey_url = 'https://vjvauction.com/rimbey/market-report/cattle'
vjv_westlock_url = 'https://vjvauction.com/westlock/market-report/cattle'
olds_url = 'https://www.oldsauction.com/market-report'
dryland_url = 'http://www.drylandcattle.com/marketreports.aspx'



In [3]:
# Read tables directly from site
okie_panokie = pd.read_html(vjv_ponoka_url)
okie_panokie

[   Click in table to get category history.             Last Auction  \
                               Week Ending:         Wed Sep 21, 2022   
 0                               Head Sold:                     3123   
 1                         Slaughter Cattle         Slaughter Cattle   
 2                             D1 - D2 Cows         $92.00 - $113.00   
 3                             D3 - D4 Cows          $78.00 - $91.00   
 4                       Good Holstein Cows          $86.00 - $92.00   
 5                     Medium Holstein Cows          $76.00 - $85.00   
 6                              Heiferettes        $120.00 - $160.00   
 7                            Bologna Bulls        $120.00 - $144.00   
 8                             Feeder Bulls        $150.00 - $185.00   
 9                       Replacement Cattle       Replacement Cattle   
 10                          Good Bred Cows                    — - —   
 11                         Older Bred Cows                    —

In [4]:
type(okie_panokie)

list

In [5]:
print(type(okie_panokie[0]))
print(len(okie_panokie))
print(len(okie_panokie[0]))

<class 'pandas.core.frame.DataFrame'>
1
36


In [6]:
# Extract df from list generated by reading in table
ponoka_df = okie_panokie[0]
ponoka_df.info()
ponoka_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 7 columns):
 #   Column                                                   Non-Null Count  Dtype 
---  ------                                                   --------------  ----- 
 0   (Click in table to get category history., Week Ending:)  36 non-null     object
 1   (Last Auction, Wed Sep 21, 2022)                         36 non-null     object
 2   (Last Auction, Wed Sep 21, 2022.1)                       36 non-null     object
 3   (Previous Auction 1, Wed Sep 14, 2022)                   36 non-null     object
 4   (Previous Auction 1, Wed Sep 14, 2022.1)                 36 non-null     object
 5   (Previous Auction 2, Wed Sep 7, 2022)                    36 non-null     object
 6   (Previous Auction 2, Wed Sep 7, 2022.1)                  36 non-null     object
dtypes: object(7)
memory usage: 2.1+ KB


Click in table to get category history.             Last Auction  \
                              Week Ending:         Wed Sep 21, 2022   
0                               Head Sold:                     3123   
1                         Slaughter Cattle         Slaughter Cattle   
2                             D1 - D2 Cows         $92.00 - $113.00   
3                             D3 - D4 Cows          $78.00 - $91.00   
4                       Good Holstein Cows          $86.00 - $92.00   
5                     Medium Holstein Cows          $76.00 - $85.00   
6                              Heiferettes        $120.00 - $160.00   
7                            Bologna Bulls        $120.00 - $144.00   
8                             Feeder Bulls        $150.00 - $185.00   
9                       Replacement Cattle       Replacement Cattle   
10                          Good Bred Cows                    — - —   
11                         Older Bred Cows                    — - —   
12                       Good Bred Heifers                    — - —   
13              Cow / Calf Pairs (Younger)                    — - —   
14                Cow / Calf Pairs (Older)                    — - —   
15                 Stock and Feeder Cattle  Stock and Feeder Cattle   
16                                  Weight                   Steers   
17               1000 lbs Steers | Heifers        $215.00 - $225.75   
18                900 lbs Steers | Heifers        $220.00 - $234.25   
19                800 lbs Steers | Heifers        $230.00 - $251.25   
20                700 lbs Steers | Heifers        $235.00 - $255.00   
21                600 lbs Steers | Heifers        $245.00 - $268.00   
22                500 lbs Steers | Heifers        $255.00 - $289.50   
23                400 lbs Steers | Heifers        $275.00 - $329.00   
24                300 lbs Steers | Heifers        $275.00 - $330.00   
25                          Dairy & Calves           Dairy & Calves   
26                            Dairy Steers        $130.00 - $160.00   
27                Baby Calves (Dairy) $/Hd                    — - —   
28                 Baby Calves (Beef) $/Hd                    — - —   
29                                    Feed                     Feed   
30                       Hay: Square Bales                    — - —   
31                        Hay: Round Bales                    — - —   
32                     Straw: Square Bales                    — - —   
33                      Straw: Round Bales                    — - —   
34                 Greenfeed: Square Bales                    — - —   
35                  Greenfeed: Round Bales                    — - —   

                                  Previous Auction 1                           \
         Wed Sep 21, 2022.1         Wed Sep 14, 2022       Wed Sep 14, 2022.1   
0                      3123                     2716                     2716   
1          Slaughter Cattle         Slaughter Cattle         Slaughter Cattle   
2          $92.00 - $113.00         $95.00 - $115.00         $95.00 - $115.00   
3           $78.00 - $91.00          $80.00 - $94.00          $80.00 - $94.00   
4           $86.00 - $92.00          $84.00 - $92.00          $84.00 - $92.00   
5           $76.00 - $85.00          $72.00 - $83.00          $72.00 - $83.00   
6         $120.00 - $160.00        $120.00 - $155.00        $120.00 - $155.00   
7         $120.00 - $144.00        $120.00 - $138.50        $120.00 - $138.50   
8         $150.00 - $185.00        $140.00 - $155.00        $140.00 - $155.00   
9        Replacement Cattle       Replacement Cattle       Replacement Cattle   
10                    — - —                    — - —                    — - —   
11                    — - —                    — - —                    — - —   
12                    — - —                    — - —                    — - —   
13                    — - —                    — - —                    — - —   
14      

In [7]:
# Stock and feeder cattle data is of interest to client, so slice down to it
ponoka_stock = ponoka_df.copy().iloc[16:25]
ponoka_stock


Click in table to get category history.       Last Auction  \
                              Week Ending:   Wed Sep 21, 2022   
16                                  Weight             Steers   
17               1000 lbs Steers | Heifers  $215.00 - $225.75   
18                900 lbs Steers | Heifers  $220.00 - $234.25   
19                800 lbs Steers | Heifers  $230.00 - $251.25   
20                700 lbs Steers | Heifers  $235.00 - $255.00   
21                600 lbs Steers | Heifers  $245.00 - $268.00   
22                500 lbs Steers | Heifers  $255.00 - $289.50   
23                400 lbs Steers | Heifers  $275.00 - $329.00   
24                300 lbs Steers | Heifers  $275.00 - $330.00   

                      Previous Auction 1                     \
   Wed Sep 21, 2022.1   Wed Sep 14, 2022 Wed Sep 14, 2022.1   
16            Heifers             Steers            Heifers   
17  $165.00 - $204.00  $210.00 - $222.75  $160.00 - $200.00   
18  $210.00 - $226.75  $220.00 - $232.25  $205.00 - $219.00   
19  $215.00 - $235.00  $230.00 - $247.00  $210.00 - $230.00   
20  $220.00 - $240.00  $240.00 - $260.50  $220.00 - $244.00   
21  $220.00 - $244.00  $250.00 - $278.00  $225.00 - $245.00   
22  $225.00 - $245.00  $275.00 - $307.00  $230.00 - $250.00   
23  $235.00 - $273.00  $275.00 - $305.00  $240.00 - $279.00   
24  $245.00 - $291.00  $250.00 - $310.00  $245.00 - $280.00   

   Previous Auction 2                     
      Wed Sep 7, 2022  Wed Sep 7, 2022.1  
16             Steers            Heifers  
17  $214.00 - $220.00  $165.00 - $190.00  
18  $210.00 - $236.50  $200.00 - $218.00  
19  $235.00 - $253.00  $215.00 - $236.00  
20  $235.00 - $257.00  $225.00 - $247.00  
21  $240.00 - $267.00  $225.00 - $247.00  
22  $245.00 - $276.00  $195.00 - $230.00  
23  $250.00 - $284.00  $200.00 - $245.00  
24  $250.00 - $305.00  $215.00 - $250.00

In [8]:
ponoka_stock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 16 to 24
Data columns (total 7 columns):
 #   Column                                                   Non-Null Count  Dtype 
---  ------                                                   --------------  ----- 
 0   (Click in table to get category history., Week Ending:)  9 non-null      object
 1   (Last Auction, Wed Sep 21, 2022)                         9 non-null      object
 2   (Last Auction, Wed Sep 21, 2022.1)                       9 non-null      object
 3   (Previous Auction 1, Wed Sep 14, 2022)                   9 non-null      object
 4   (Previous Auction 1, Wed Sep 14, 2022.1)                 9 non-null      object
 5   (Previous Auction 2, Wed Sep 7, 2022)                    9 non-null      object
 6   (Previous Auction 2, Wed Sep 7, 2022.1)                  9 non-null      object
dtypes: object(7)
memory usage: 636.0+ bytes


In [9]:
print(type(ponoka_stock.columns))
print(ponoka_stock.columns[0])
ponoka_stock.columns

<class 'pandas.core.indexes.multi.MultiIndex'>
('Click in table to get category history.', 'Week\xa0Ending:')


MultiIndex([('Click in table to get category history.',       'Week Ending:'),
            (                           'Last Auction',   'Wed Sep 21, 2022'),
            (                           'Last Auction', 'Wed Sep 21, 2022.1'),
            (                     'Previous Auction 1',   'Wed Sep 14, 2022'),
            (                     'Previous Auction 1', 'Wed Sep 14, 2022.1'),
            (                     'Previous Auction 2',    'Wed Sep 7, 2022'),
            (                     'Previous Auction 2',  'Wed Sep 7, 2022.1')],
           )

In [10]:
print(type(ponoka_stock[:0]))
print(len(ponoka_stock[:0]))
print(type(ponoka_stock[:0].T))
print(len(ponoka_stock[:0].T))
ponoka_stock[:0].T.info()
ponoka_stock[:0].T

<class 'pandas.core.frame.DataFrame'>
0
<class 'pandas.core.frame.DataFrame'>
7
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 7 entries, ('Click in table to get category history.', 'Week\xa0Ending:') to ('Previous Auction 2', 'Wed Sep 7, 2022.1')
Empty DataFrame

Empty DataFrame
Columns: []
Index: [(Click in table to get category history., Week Ending:), (Last Auction, Wed Sep 21, 2022), (Last Auction, Wed Sep 21, 2022.1), (Previous Auction 1, Wed Sep 14, 2022), (Previous Auction 1, Wed Sep 14, 2022.1), (Previous Auction 2, Wed Sep 7, 2022), (Previous Auction 2, Wed Sep 7, 2022.1)]

In [11]:
# Transpose df to position named rows as columns
ponoka_df = ponoka_df.T
ponoka_df

ponoka_stock = ponoka_stock.T
ponoka_stock

16  \
Click in table to get category history. Week Ending:         Weight   
Last Auction                            Wed Sep 21, 2022     Steers   
                                        Wed Sep 21, 2022.1  Heifers   
Previous Auction 1                      Wed Sep 14, 2022     Steers   
                                        Wed Sep 14, 2022.1  Heifers   
Previous Auction 2                      Wed Sep 7, 2022      Steers   
                                        Wed Sep 7, 2022.1   Heifers   

                                                                                   17  \
Click in table to get category history. Week Ending:        1000 lbs Steers | Heifers   
Last Auction                            Wed Sep 21, 2022            $215.00 - $225.75   
                                        Wed Sep 21, 2022.1          $165.00 - $204.00   
Previous Auction 1                      Wed Sep 14, 2022            $210.00 - $222.75   
                                        Wed Sep 14, 2022.1          $160.00 - $200.00   
Previous Auction 2                      Wed Sep 7, 2022             $214.00 - $220.00   
                                        Wed Sep 7, 2022.1           $165.00 - $190.00   

                                                                                  18  \
Click in table to get category history. Week Ending:        900 lbs Steers | Heifers   
Last Auction                            Wed Sep 21, 2022           $220.00 - $234.25   
                                        Wed Sep 21, 2022.1         $210.00 - $226.75   
Previous Auction 1                      Wed Sep 14, 2022           $220.00 - $232.25   
                                        Wed Sep 14, 2022.1         $205.00 - $219.00   
Previous Auction 2                      Wed Sep 7, 2022            $210.00 - $236.50   
                                        Wed Sep 7, 2022.1          $200.00 - $218.00   

                                                                                  19  \
Click in table to get category history. Week Ending:        800 lbs Steers | Heifers   
Last Auction                            Wed Sep 21, 2022           $230.00 - $251.25   
                                        Wed Sep 21, 2022.1         $215.00 - $235.00   
Previous Auction 1                      Wed Sep 14, 2022           $230.00 - $247.00   
                                        Wed Sep 14, 2022.1         $210.00 - $230.00   
Previous Auction 2                      Wed Sep 7, 2022            $235.00 - $253.00   
                                        Wed Sep 7, 2022.1          $215.00 - $236.00   

                                                                                  20  \
Click in table to get category history. Week Ending:        700 lbs Steers | Heifers   
Last Auction                            Wed Sep 21, 2022           $235.00 - $255.00   
                                        Wed Sep 21, 2022.1         $220.00 - $240.00   
Previous Auction 1                      Wed Sep 14, 2022           $240.00 - $260.50   
                                        Wed Sep 14, 2022.1         $220.00 - $244.00   
Previous Auction 2                      Wed Sep 7, 2022            $235.00 - $257.00   
                                        Wed Sep 7, 2022.1          $225.00 - $247.00   

                                                                                  21  \
Click in table to get category history. Week Ending:        600 lbs Steers | Heifers   
Last Auction                            Wed Sep 21, 2022           $245.00 - $268.00   
                                        Wed Sep 21, 2022.1         $220.00 - $244.00   
Previous Auction 1                      Wed Sep 14, 2022           $250.00 - $278.00   
                                        Wed Sep 14, 2022.1         $225.00 - $245.00   
Previous Auction 2                      Wed Sep 7, 2022            $240.00 - $267.00   
                                        Wed Sep 7, 2

In [11]:
# Isolate columns from row
ponoka_stock.iloc[0]

Click in table to get category history.  Week Ending:           Weight
Last Auction                             Wed Sep 21, 2022       Steers
                                         Wed Sep 21, 2022.1    Heifers
Previous Auction 1                       Wed Sep 14, 2022       Steers
                                         Wed Sep 14, 2022.1    Heifers
Previous Auction 2                       Wed Sep 7, 2022        Steers
                                         Wed Sep 7, 2022.1     Heifers
Name: 16, dtype: object

In [12]:
# Set column names to first row
ponoka_stock.columns = ponoka_stock.iloc[0]
# Rename columns to Python 
ponoka_stock.columns = [col.lower()\
    .replace(' | ', '_')\
        .replace(' ', '_') for col in ponoka_stock.columns]

ponoka_stock = ponoka_stock.rename(columns = {'weight' : 'sex'})
ponoka_stock = ponoka_stock[1:]

ponoka_stock

,sex,steers,heifers,steers,heifers,steers,heifers
17,1000 lbs Steers | Heifers,$215.00 - $225.75,$165.00 - $204.00,$210.00 - $222.75,$160.00 - $200.00,$214.00 - $220.00,$165.00 - $190.00
18,900 lbs Steers | Heifers,$220.00 - $234.25,$210.00 - $226.75,$220.00 - $232.25,$205.00 - $219.00,$210.00 - $236.50,$200.00 - $218.00
19,800 lbs Steers | Heifers,$230.00 - $251.25,$215.00 - $235.00,$230.00 - $247.00,$210.00 - $230.00,$235.00 - $253.00,$215.00 - $236.00
20,700 lbs Steers | Heifers,$235.00 - $255.00,$220.00 - $240.00,$240.00 - $260.50,$220.00 - $244.00,$235.00 - $257.00,$225.00 - $247.00
21,600 lbs Steers | Heifers,$245.00 - $268.00,$220.00 - $244.00,$250.00 - $278.00,$225.00 - $245.00,$240.00 - $267.00,$225.00 - $247.00
22,500 lbs Steers | Heifers,$255.00 - $289.50,$225.00 - $245.00,$275.00 - $307.00,$230.00 - $250.00,$245.00 - $276.00,$195.00 - $230.00
23,400 lbs Steers | Heifers,$275.00 - $329.00,$235.00 - $273.00,$275.00 - $305.00,$240.00 - $279.00,$250.00 - $284.00,$200.00 - $245.00
24,300 lbs Steers | Heifers,$275.00 - $330.00,$245.00 - $291.00,$250.00 - $310.00,$245.00 - $280.00,$250.00 - $305.00,$215.00 - $250.00


In [17]:
print(type(ponoka_stock.index))

flat_ps_idx = ponoka_stock.index.to_flat_index()
print(type(flat_ps_idx))
flat_ps_idx
ponoka_stock1 = ponoka_stock.copy().set_index(flat_ps_idx)
ponoka_stock1

<class 'pandas.core.indexes.multi.MultiIndex'>
<class 'pandas.core.indexes.base.Index'>


,sex,1000_lbs_steers_heifers,900_lbs_steers_heifers,800_lbs_steers_heifers,700_lbs_steers_heifers,600_lbs_steers_heifers,500_lbs_steers_heifers,400_lbs_steers_heifers,300_lbs_steers_heifers
"(Last Auction, Wed Sep 21, 2022)",Steers,$215.00 - $225.75,$220.00 - $234.25,$230.00 - $251.25,$235.00 - $255.00,$245.00 - $268.00,$255.00 - $289.50,$275.00 - $329.00,$275.00 - $330.00
"(Last Auction, Wed Sep 21, 2022.1)",Heifers,$165.00 - $204.00,$210.00 - $226.75,$215.00 - $235.00,$220.00 - $240.00,$220.00 - $244.00,$225.00 - $245.00,$235.00 - $273.00,$245.00 - $291.00
"(Previous Auction 1, Wed Sep 14, 2022)",Steers,$210.00 - $222.75,$220.00 - $232.25,$230.00 - $247.00,$240.00 - $260.50,$250.00 - $278.00,$275.00 - $307.00,$275.00 - $305.00,$250.00 - $310.00
"(Previous Auction 1, Wed Sep 14, 2022.1)",Heifers,$160.00 - $200.00,$205.00 - $219.00,$210.00 - $230.00,$220.00 - $244.00,$225.00 - $245.00,$230.00 - $250.00,$240.00 - $279.00,$245.00 - $280.00
"(Previous Auction 2, Wed Sep 7, 2022)",Steers,$214.00 - $220.00,$210.00 - $236.50,$235.00 - $253.00,$235.00 - $257.00,$240.00 - $267.00,$245.00 - $276.00,$250.00 - $284.00,$250.00 - $305.00
"(Previous Auction 2, Wed Sep 7, 2022.1)",Heifers,$165.00 - $190.00,$200.00 - $218.00,$215.00 - $236.00,$225.00 - $247.00,$225.00 - $247.00,$195.00 - $230.00,$200.00 - $245.00,$215.00 - $250.00


In [ ]:
ponoka_stock1['thousand_lbs_range'] = ponoka_stock1['1000_lbs_steers_heifers'].str.replace('$', '')
ponoka_stock1.drop(columns= ['1000_lbs_range'], inplace=True)


In [26]:
ponoka_stock1.thousand_lbs_range = ponoka_stock1.thousand_lbs_range.str.replace(' -', ',')
ponoka_stock1.thousand_lbs_range

(Last Auction, Wed Sep 21, 2022)            215.00 225.75
(Last Auction, Wed Sep 21, 2022.1)          165.00 204.00
(Previous Auction 1, Wed Sep 14, 2022)      210.00 222.75
(Previous Auction 1, Wed Sep 14, 2022.1)    160.00 200.00
(Previous Auction 2, Wed Sep 7, 2022)       214.00 220.00
(Previous Auction 2, Wed Sep 7, 2022.1)     165.00 190.00
Name: thousand_lbs_range, dtype: object

In [12]:
type(ponoka_df.iloc[0])

pandas.core.series.Series

In [13]:
# Set row as columns
ponoka_df.columns = ponoka_df.iloc[0]
# Normalize column names
ponoka_df.columns = [col.lower()\
    .replace(' ', '_')\
        .replace('_/_', '_')\
            .replace('-', 'to')\
                .replace('_|_', '_')\
                    .replace('$', 'cad')\
                        .replace(':', '')\
                            .replace('(', '')\
                                .replace(')', '') for col in ponoka_df.columns]
ponoka_df.head()

head_sold  \
Click in table to get category history. Week Ending:        Head Sold:   
Last Auction                            Wed Sep 14, 2022          2716   
                                        Wed Sep 14, 2022.1        2716   
Previous Auction 1                      Wed Sep 7, 2022           1476   
                                        Wed Sep 7, 2022.1         1476   

                                                            slaughter_cattle  \
Click in table to get category history. Week Ending:        Slaughter Cattle   
Last Auction                            Wed Sep 14, 2022    Slaughter Cattle   
                                        Wed Sep 14, 2022.1  Slaughter Cattle   
Previous Auction 1                      Wed Sep 7, 2022     Slaughter Cattle   
                                        Wed Sep 7, 2022.1   Slaughter Cattle   

                                                               d1_to_d2_cows  \
Click in table to get category history. Week Ending:            D1 - D2 Cows   
Last Auction                            Wed Sep 14, 2022    $95.00 - $115.00   
                                        Wed Sep 14, 2022.1  $95.00 - $115.00   
Previous Auction 1                      Wed Sep 7, 2022     $94.00 - $114.00   
                                        Wed Sep 7, 2022.1   $94.00 - $114.00   

                                                              d3_to_d4_cows  \
Click in table to get category history. Week Ending:           D3 - D4 Cows   
Last Auction                            Wed Sep 14, 2022    $80.00 - $94.00   
                                        Wed Sep 14, 2022.1  $80.00 - $94.00   
Previous Auction 1                      Wed Sep 7, 2022     $80.00 - $93.00   
                                        Wed Sep 7, 2022.1   $80.00 - $93.00   

                                                            good_holstein_cows  \
Click in table to get category history. Week Ending:        Good Holstein Cows   
Last Auction                            Wed Sep 14, 2022       $84.00 - $92.00   
                                        Wed Sep 14, 2022.1     $84.00 - $92.00   
Previous Auction 1                      Wed Sep 7, 2022        $83.00 - $88.00   
                                        Wed Sep 7, 2022.1      $83.00 - $88.00   

                                                            medium_holstein_cows  \
Click in table to get category history. Week Ending:        Medium Holstein Cows   
Last Auction                            Wed Sep 14, 2022         $72.00 - $83.00   
                                        Wed Sep 14, 2022.1       $72.00 - $83.00   
Previous Auction 1                      Wed Sep 7, 2022          $72.00 - $82.00   
                                        Wed Sep 7, 2022.1        $72.00 - $82.00   

                                                                  heiferettes  \
Click in table to get category history. Week Ending:              Heiferettes   
Last Auction                            Wed Sep 14, 2022    $120.00 - $155.00   
                                        Wed Sep 14, 2022.1  $120.00 - $155.00   
Previous Auction 1                      Wed Sep 7, 2022     $120.00 - $155.00   
                                        Wed Sep 7, 2022.1   $120.00 - $155.00   

                                                                bologna_bulls  \
Click in table to get category history. Week Ending:            Bologna Bulls   
Last Auction                            Wed Sep 14, 2022    $120.00 - $138.50   
                                        Wed Sep 14, 2022.1  $120.00 - $138.50   
Previous Auction 1                      Wed Sep 7, 2022     $110.00 - $138.50   
                                        Wed Sep 7, 2022.1   $110.00 - $138.50   

                                                                 feeder_bulls  \
Click in table to get category history. Week Ending:             Feeder Bulls   
Last Auction                            Wed S

In [14]:
ponoka_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 7 entries, ('Click in table to get category history.', 'Week\xa0Ending:') to ('Previous Auction 2', 'Wed Aug 31, 2022.1')
Data columns (total 36 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   head_sold                 7 non-null      object
 1   slaughter_cattle          7 non-null      object
 2   d1_to_d2_cows             7 non-null      object
 3   d3_to_d4_cows             7 non-null      object
 4   good_holstein_cows        7 non-null      object
 5   medium_holstein_cows      7 non-null      object
 6   heiferettes               7 non-null      object
 7   bologna_bulls             7 non-null      object
 8   feeder_bulls              7 non-null      object
 9   replacement_cattle        7 non-null      object
 10  good_bred_cows            7 non-null      object
 11  older_bred_cows           7 non-null      object
 12  good_bred_heifers         7 non-nu

In [15]:
# Drop first row and check df
ponoka_df = ponoka_df.iloc[1:]
ponoka_df.info()
ponoka_df.head()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 6 entries, ('Last Auction', 'Wed Sep 14, 2022') to ('Previous Auction 2', 'Wed Aug 31, 2022.1')
Data columns (total 36 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   head_sold                 6 non-null      object
 1   slaughter_cattle          6 non-null      object
 2   d1_to_d2_cows             6 non-null      object
 3   d3_to_d4_cows             6 non-null      object
 4   good_holstein_cows        6 non-null      object
 5   medium_holstein_cows      6 non-null      object
 6   heiferettes               6 non-null      object
 7   bologna_bulls             6 non-null      object
 8   feeder_bulls              6 non-null      object
 9   replacement_cattle        6 non-null      object
 10  good_bred_cows            6 non-null      object
 11  older_bred_cows           6 non-null      object
 12  good_bred_heifers         6 non-null      object
 13  cow_ca

head_sold  slaughter_cattle  \
Last Auction       Wed Sep 14, 2022        2716  Slaughter Cattle   
                   Wed Sep 14, 2022.1      2716  Slaughter Cattle   
Previous Auction 1 Wed Sep 7, 2022         1476  Slaughter Cattle   
                   Wed Sep 7, 2022.1       1476  Slaughter Cattle   
Previous Auction 2 Wed Aug 31, 2022        3529  Slaughter Cattle   

                                           d1_to_d2_cows    d3_to_d4_cows  \
Last Auction       Wed Sep 14, 2022     $95.00 - $115.00  $80.00 - $94.00   
                   Wed Sep 14, 2022.1   $95.00 - $115.00  $80.00 - $94.00   
Previous Auction 1 Wed Sep 7, 2022      $94.00 - $114.00  $80.00 - $93.00   
                   Wed Sep 7, 2022.1    $94.00 - $114.00  $80.00 - $93.00   
Previous Auction 2 Wed Aug 31, 2022    $100.00 - $120.00  $85.00 - $99.00   

                                      good_holstein_cows medium_holstein_cows  \
Last Auction       Wed Sep 14, 2022      $84.00 - $92.00      $72.00 - $83.00   
                   Wed Sep 14, 2022.1    $84.00 - $92.00      $72.00 - $83.00   
Previous Auction 1 Wed Sep 7, 2022       $83.00 - $88.00      $72.00 - $82.00   
                   Wed Sep 7, 2022.1     $83.00 - $88.00      $72.00 - $82.00   
Previous Auction 2 Wed Aug 31, 2022      $88.00 - $94.00      $74.00 - $87.00   

                                             heiferettes      bologna_bulls  \
Last Auction       Wed Sep 14, 2022    $120.00 - $155.00  $120.00 - $138.50   
                   Wed Sep 14, 2022.1  $120.00 - $155.00  $120.00 - $138.50   
Previous Auction 1 Wed Sep 7, 2022     $120.00 - $155.00  $110.00 - $138.50   
                   Wed Sep 7, 2022.1   $120.00 - $155.00  $110.00 - $138.50   
Previous Auction 2 Wed Aug 31, 2022    $120.00 - $155.00  $115.00 - $155.00   

                                            feeder_bulls  replacement_cattle  \
Last Auction       Wed Sep 14, 2022    $140.00 - $155.00  Replacement Cattle   
                   Wed Sep 14, 2022.1  $140.00 - $155.00  Replacement Cattle   
Previous Auction 1 Wed Sep 7, 2022     $120.00 - $150.00  Replacement Cattle   
                   Wed Sep 7, 2022.1   $120.00 - $150.00  Replacement Cattle   
Previous Auction 2 Wed Aug 31, 2022    $125.00 - $160.00  Replacement Cattle   

                                      good_bred_cows older_bred_cows  \
Last Auction       Wed Sep 14, 2022            — - —           — - —   
                   Wed Sep 14, 2022.1          — - —           — - —   
Previous Auction 1 Wed Sep 7, 2022             — - —           — - —   
                   Wed Sep 7, 2022.1           — - —           — - —   
Previous Auction 2 Wed Aug 31, 2022            — - —           — - —   

                                      good_bred_heifers  \
Last Auction       Wed Sep 14, 2022               — - —   
                   Wed Sep 14, 2022.1             — - —   
Previous Auction 1 Wed Sep 7, 2022                — - —   
                   Wed Sep 7, 2022.1              — - —   
Previous Auction 2 Wed Aug 31, 2022               — - —   

                                      cow_calf_pairs_younger  \
Last Auction       Wed Sep 14, 2022                    — - —   
                   Wed Sep 14, 2022.1                  — - —   
Previous Auction 1 Wed Sep 7, 2022                     — - —   
                   Wed Sep 7, 2022.1                   — - —   
Previous Auction 2 Wed Aug 31, 2022                    — - —   

                                      cow_calf_pairs_older  \
Last Auction       Wed Sep 14, 2022                  — - —   
                   Wed Sep 14, 2022.1                — - —   
Previous Auction 1 Wed Sep 7, 2022                   — - —   
                   Wed Sep 7, 2022.1                 — - —   
Previous Auction 2 Wed Aug 31, 2022                  — - —   

                                       stock_and_feeder_cattle   weight  \
Last Auction       Wed Sep 14, 2022    Stock and Feeder Cattle   Steers   
                   We

In [16]:
print(ponoka_df.index)
type(ponoka_df.index)

MultiIndex([(      'Last Auction',   'Wed Sep 14, 2022'),
            (      'Last Auction', 'Wed Sep 14, 2022.1'),
            ('Previous Auction 1',    'Wed Sep 7, 2022'),
            ('Previous Auction 1',  'Wed Sep 7, 2022.1'),
            ('Previous Auction 2',   'Wed Aug 31, 2022'),
            ('Previous Auction 2', 'Wed Aug 31, 2022.1')],
           )


pandas.core.indexes.multi.MultiIndex

In [17]:
dt_idx = ponoka_df.index.get_level_values(1)
ponoka_df = ponoka_df.set_index(dt_idx)
ponoka_df

,head_sold,slaughter_cattle,d1_to_d2_cows,d3_to_d4_cows,good_holstein_cows,medium_holstein_cows,heiferettes,bologna_bulls,feeder_bulls,replacement_cattle,good_bred_cows,older_bred_cows,good_bred_heifers,cow_calf_pairs_younger,cow_calf_pairs_older,stock_and_feeder_cattle,weight,1000_lbs_steers_heifers,900_lbs_steers_heifers,800_lbs_steers_heifers,700_lbs_steers_heifers,600_lbs_steers_heifers,500_lbs_steers_heifers,400_lbs_steers_heifers,300_lbs_steers_heifers,dairy_&_calves,dairy_steers,baby_calves_dairy_cad/hd,baby_calves_beef_cad/hd,feed,hay_square_bales,hay_round_bales,straw_square_bales,straw_round_bales,greenfeed_square_bales,greenfeed_round_bales
"Wed Sep 14, 2022",2716,Slaughter Cattle,$95.00 - $115.00,$80.00 - $94.00,$84.00 - $92.00,$72.00 - $83.00,$120.00 - $155.00,$120.00 - $138.50,$140.00 - $155.00,Replacement Cattle,— - —,— - —,— - —,— - —,— - —,Stock and Feeder Cattle,Steers,$210.00 - $222.75,$220.00 - $232.25,$230.00 - $247.00,$240.00 - $260.50,$250.00 - $278.00,$275.00 - $307.00,$275.00 - $305.00,$250.00 - $310.00,Dairy & Calves,$135.00 - $165.00,— - —,— - —,Feed,— - $5.00,— - $120.00,— - —,— - —,— - —,— - —
"Wed Sep 14, 2022.1",2716,Slaughter Cattle,$95.00 - $115.00,$80.00 - $94.00,$84.00 - $92.00,$72.00 - $83.00,$120.00 - $155.00,$120.00 - $138.50,$140.00 - $155.00,Replacement Cattle,— - —,— - —,— - —,— - —,— - —,Stock and Feeder Cattle,Heifers,$160.00 - $200.00,$205.00 - $219.00,$210.00 - $230.00,$220.00 - $244.00,$225.00 - $245.00,$230.00 - $250.00,$240.00 - $279.00,$245.00 - $280.00,Dairy & Calves,$135.00 - $165.00,— - —,— - —,Feed,— - $5.00,— - $120.00,— - —,— - —,— - —,— - —
"Wed Sep 7, 2022",1476,Slaughter Cattle,$94.00 - $114.00,$80.00 - $93.00,$83.00 - $88.00,$72.00 - $82.00,$120.00 - $155.00,$110.00 - $138.50,$120.00 - $150.00,Replacement Cattle,— - —,— - —,— - —,— - —,— - —,Stock and Feeder Cattle,Steers,$214.00 - $220.00,$210.00 - $236.50,$235.00 - $253.00,$235.00 - $257.00,$240.00 - $267.00,$245.00 - $276.00,$250.00 - $284.00,$250.00 - $305.00,Dairy & Calves,$120.00 - $160.00,— - —,— - —,Feed,— - —,— - —,— - $2.50,— - —,— - —,— - —
"Wed Sep 7, 2022.1",1476,Slaughter Cattle,$94.00 - $114.00,$80.00 - $93.00,$83.00 - $88.00,$72.00 - $82.00,$120.00 - $155.00,$110.00 - $138.50,$120.00 - $150.00,Replacement Cattle,— - —,— - —,— - —,— - —,— - —,Stock and Feeder Cattle,Heifers,$165.00 - $190.00,$200.00 - $218.00,$215.00 - $236.00,$225.00 - $247.00,$225.00 - $247.00,$195.00 - $230.00,$200.00 - $245.00,$215.00 - $250.00,Dairy & Calves,$120.00 - $160.00,— - —,— - —,Feed,— - —,— - —,— - $2.50,— - —,— - —,— - —
"Wed Aug 31, 2022",3529,Slaughter Cattle,$100.00 - $120.00,$85.00 - $99.00,$88.00 - $94.00,$74.00 - $87.00,$120.00 - $155.00,$115.00 - $155.00,$125.00 - $160.00,Replacement Cattle,— - —,— - —,— - —,— - —,— - —,Stock and Feeder Cattle,Steers,$215.00 - $226.75,$225.00 - $241.50,$230.00 - $253.00,$240.00 - $264.50,$245.00 - $265.50,$245.00 - $270.00,$230.00 - $288.00,$230.00 - $282.50,Dairy & Calves,$125.00 - $155.00,— - —,— - —,Feed,— - —,— - —,— - —,— - —,— - —,— - —
"Wed Aug 31, 2022.1",3529,Slaughter Cattle,$100.00 - $120.00,$85.00 - $99.00,$88.00 - $94.00,$74.00 - $87.00,$120.00 - $155.00,$115.00 - $155.00,$125.00 - $160.00,Replacement Cattle,— - —,— - —,— - —,— - —,— - —,Stock and Feeder Cattle,Heifers,$170.00 - $203.00,$195.00 - $219.50,$210.00 - $234.50,$225.00 - $248.00,$230.00 - $259.50,$200.00 - $236.00,$205.00 - $240.00,$210.00 - $240.00,Dairy & Calves,$125.00 - $155.00,— - —,— - —,Feed,— - —,— - —,— - —,— - —,— - —,— - —


In [21]:
ponoka_df.drop(['Wed Sep 14, 2022.1', 'Wed Sep 7, 2022.1', 'Wed Aug 31, 2022.1'])

rgx = r'.\.1'
rows_to_drop = []


In [24]:
print(ponoka_df.index.str.contains(rgx))


[False  True False  True False  True]


In [25]:
# Dropping duplicate rows from df
ponoka_df = ponoka_df[ponoka_df.index.str.contains(rgx) == False]
ponoka_df

,head_sold,slaughter_cattle,d1_to_d2_cows,d3_to_d4_cows,good_holstein_cows,medium_holstein_cows,heiferettes,bologna_bulls,feeder_bulls,replacement_cattle,good_bred_cows,older_bred_cows,good_bred_heifers,cow_calf_pairs_younger,cow_calf_pairs_older,stock_and_feeder_cattle,weight,1000_lbs_steers_heifers,900_lbs_steers_heifers,800_lbs_steers_heifers,700_lbs_steers_heifers,600_lbs_steers_heifers,500_lbs_steers_heifers,400_lbs_steers_heifers,300_lbs_steers_heifers,dairy_&_calves,dairy_steers,baby_calves_dairy_cad/hd,baby_calves_beef_cad/hd,feed,hay_square_bales,hay_round_bales,straw_square_bales,straw_round_bales,greenfeed_square_bales,greenfeed_round_bales
"Wed Sep 14, 2022",2716,Slaughter Cattle,$95.00 - $115.00,$80.00 - $94.00,$84.00 - $92.00,$72.00 - $83.00,$120.00 - $155.00,$120.00 - $138.50,$140.00 - $155.00,Replacement Cattle,— - —,— - —,— - —,— - —,— - —,Stock and Feeder Cattle,Steers,$210.00 - $222.75,$220.00 - $232.25,$230.00 - $247.00,$240.00 - $260.50,$250.00 - $278.00,$275.00 - $307.00,$275.00 - $305.00,$250.00 - $310.00,Dairy & Calves,$135.00 - $165.00,— - —,— - —,Feed,— - $5.00,— - $120.00,— - —,— - —,— - —,— - —
"Wed Sep 7, 2022",1476,Slaughter Cattle,$94.00 - $114.00,$80.00 - $93.00,$83.00 - $88.00,$72.00 - $82.00,$120.00 - $155.00,$110.00 - $138.50,$120.00 - $150.00,Replacement Cattle,— - —,— - —,— - —,— - —,— - —,Stock and Feeder Cattle,Steers,$214.00 - $220.00,$210.00 - $236.50,$235.00 - $253.00,$235.00 - $257.00,$240.00 - $267.00,$245.00 - $276.00,$250.00 - $284.00,$250.00 - $305.00,Dairy & Calves,$120.00 - $160.00,— - —,— - —,Feed,— - —,— - —,— - $2.50,— - —,— - —,— - —
"Wed Aug 31, 2022",3529,Slaughter Cattle,$100.00 - $120.00,$85.00 - $99.00,$88.00 - $94.00,$74.00 - $87.00,$120.00 - $155.00,$115.00 - $155.00,$125.00 - $160.00,Replacement Cattle,— - —,— - —,— - —,— - —,— - —,Stock and Feeder Cattle,Steers,$215.00 - $226.75,$225.00 - $241.50,$230.00 - $253.00,$240.00 - $264.50,$245.00 - $265.50,$245.00 - $270.00,$230.00 - $288.00,$230.00 - $282.50,Dairy & Calves,$125.00 - $155.00,— - —,— - —,Feed,— - —,— - —,— - —,— - —,— - —,— - —


In [35]:
# Columns of interest: Steers & Heifers 300-1000 lbs
ponoka_steers = ponoka_df.copy()[[
    'weight',
    '1000_lbs_steers_heifers', 
    '900_lbs_steers_heifers',
    '800_lbs_steers_heifers',
    '700_lbs_steers_heifers',
    '600_lbs_steers_heifers',
    '500_lbs_steers_heifers',
    '400_lbs_steers_heifers',
    '300_lbs_steers_heifers']]
    
ponoka_steers



,weight,1000_lbs_steers_heifers,900_lbs_steers_heifers,800_lbs_steers_heifers,700_lbs_steers_heifers,600_lbs_steers_heifers,500_lbs_steers_heifers,400_lbs_steers_heifers,300_lbs_steers_heifers
"Wed Sep 14, 2022",Steers,$210.00 - $222.75,$220.00 - $232.25,$230.00 - $247.00,$240.00 - $260.50,$250.00 - $278.00,$275.00 - $307.00,$275.00 - $305.00,$250.00 - $310.00
"Wed Sep 7, 2022",Steers,$214.00 - $220.00,$210.00 - $236.50,$235.00 - $253.00,$235.00 - $257.00,$240.00 - $267.00,$245.00 - $276.00,$250.00 - $284.00,$250.00 - $305.00
"Wed Aug 31, 2022",Steers,$215.00 - $226.75,$225.00 - $241.50,$230.00 - $253.00,$240.00 - $264.50,$245.00 - $265.50,$245.00 - $270.00,$230.00 - $288.00,$230.00 - $282.50
